#load 



#check alternative
https://github.com/langchain-ai/langchain/blob/master/cookbook/Semi_Structured_RAG.ipynb

In [1]:
import os

api_key=os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_community.document_loaders import DirectoryLoader

In [30]:
docs_dir="data/"
chunk_size=5000
chunk_overlap=100
file_extension="pdf"
docs_glob: str = f"**/*.{file_extension}"
qdrant_url="http://localhost:6333/"
embedding_model="text-embedding-ada-002"
qdrant_collection="recetas"


#Load all the data of a directory
Under the hood, by default this uses the UnstructuredLoader.
Ref. https://python.langchain.com/docs/modules/data_connection/document_loaders/file_directory/


In [4]:
loader = DirectoryLoader(docs_dir, docs_glob, show_progress=True)
docs = loader.load()
len(docs)


This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSe

1

In [12]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    is_separator_regex=False,
)

data=text_splitter.split_documents(docs)

In [13]:
len(data)

68

In [16]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
            openai_api_key=api_key
        )


In [31]:
from langchain_community.vectorstores import Qdrant
Qdrant.from_documents(
    data,
    embeddings,
    url="http://localhost:6333/",
    api_key="",
    port=None,
    collection_name=qdrant_collection
)

/home/mcastrol/github/openllmmodels/.venv/lib/python3.11/site-packages/qdrant_client/qdrant_remote.py:123: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")
/home/mcastrol/github/openllmmodels/.venv/lib/python3.11/site-packages/qdrant_client/async_qdrant_remote.py:112: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")


#ask some questions
    

In [19]:
questions=[
    "Como se hace el pan de platano",
    "Como se hace el cereal de semillas nutritivas",
]

In [28]:
encoded_query = embeddings.embed_query(questions[0])

In [33]:
from qdrant_client import QdrantClient


qdrant_client = QdrantClient(url=qdrant_url,
                             api_key="")

result = qdrant_client.search(
        collection_name=qdrant_collection,
        query_vector=encoded_query,
        limit=2, score_threshold=0.50
    )

/home/mcastrol/github/openllmmodels/.venv/lib/python3.11/site-packages/qdrant_client/qdrant_remote.py:123: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")


In [34]:
result

[ScoredPoint(id='2d60d298-7b4e-49c2-bccf-44548829c65c', version=0, score=0.8781653, payload={'metadata': {'source': 'data/Recetario+Nutre+tu+Cuerpo.pdf'}, 'page_content': '3. Extender la mezcla en un molde para hornear hasta que se distribuya bien, presionar con una cuchara.\n\n2 platanos maduros.\n\n4 cucharadas de manteauilla de almenara.\n\nYa de taza de cacao en polvo sin azucar.\n\n4 cucharadas de miel.\n\n4. Mezclar pldtanos, mantequilla de almendra, miel y el otro taza de cacao en polvo en el procesador.\n\n5, Extender esta mezcla sobre la capa inferior hasta distriouirla bien.\n\n6. Colocar en el congelador 30 minutos Cortar 16 cuadros y re-congelar.\n\nSushi de platano con crema de almendra\n\nINGREDIENTES\n\n% sirve a tu eleccién\n\n* 1 platano.\n\n+ 2 cucharadas de crema de almendra o de cualquier otra nuez.\n\n* 2 cucharadas de semillas de hemp.\n\nPREPARACION\n\n].Pela el pldtano y unta la crema de almendra en la mitad superior del pldtano.\n\n2. Espolvorea las semillas de